In [4]:
import sys
import json
from pathlib import Path
import numpy as np
import pandas as pd
import requests
from sklearn.metrics import roc_auc_score
sys.path.append("..")
from src.data_prep import build_datasets
from src.metrics import business_cost

train_df, test_df = build_datasets()
X = train_df.drop(columns=["TARGET"])
y = train_df["TARGET"]
SAMPLE_SIZE = 1000
X_sample = X.sample(n=SAMPLE_SIZE, random_state=42)
y_sample = y.loc[X_sample.index]

print("Sample shape:", X_sample.shape)
print("Default rate in sample:", y_sample.mean())

Sample shape: (1000, 192)
Default rate in sample: 0.093


In [10]:
import json
import requests
import pandas as pd
import numpy as np
import sys

sys.path.append("..")
from src.data_prep import build_datasets

train_df, test_df = build_datasets()
X_test = test_df.copy()

X_sample = X_test.sample(5, random_state=42)
print("Sample shape:", X_sample.shape)

X_sample_clean = X_sample.replace([np.nan, np.inf, -np.inf], 0.0)

payload = {
    "dataframe_split": X_sample_clean.to_dict(orient="split")
}

url = "http://127.0.0.1:1234/invocations"

headers = {"Content-Type": "application/json"}

response = requests.post(
    url,
    headers=headers,
    data=json.dumps(payload),
)

print("HTTP status code:", response.status_code)

if response.status_code != 200:
    print("API error:", response.text)
else:
    print("Predictions from server:", response.json())


Sample shape: (5, 192)
HTTP status code: 200
Predictions from server: {'predictions': [1, 1, 0, 0, 1]}


In [11]:

resp_json = response.json()
if isinstance(resp_json, dict) and "predictions" in resp_json:
    preds = np.array(resp_json["predictions"])
else:
    preds = np.array(resp_json)

print("Predictions shape:", preds.shape)
print("First 10 predictions:", preds[:10])


Predictions shape: (5,)
First 10 predictions: [1 1 0 0 1]


In [12]:
y_pred = preds.astype(int)

auc_api = roc_auc_score(y_sample, y_pred)
y_scores = preds.astype(float)

cost_api, conf_api = business_cost(
    y_true=y_sample,
    y_proba=y_scores,
    threshold=0.5,
    cost_fn=10.0,
    cost_fp=1.0,
    normalize=True,
)

print(f"AUC (API) : {auc_api:.4f}")
print(f"Business cost (API, seuil 0.5) : {cost_api:.4f}")
print("Confusion matrix (tn, fp, fn, tp):", conf_api)


ValueError: Found input variables with inconsistent numbers of samples: [1000, 5]

In [9]:
one_row = X_sample.iloc[[0]]
curl_payload = {
    "dataframe_split": one_row.to_dict(orient="split")
}

curl_cmd = (
    "curl -X POST http://127.0.0.1:1234/invocations "
    "-H 'Content-Type: application/json' "
    f"--data '{json.dumps(curl_payload)}'"
)

print("Example curl command:\n")
print(curl_cmd)


Example curl command:

curl -X POST http://127.0.0.1:1234/invocations -H 'Content-Type: application/json' --data '{"dataframe_split": {"index": [245895], "columns": ["SK_ID_CURR", "NAME_CONTRACT_TYPE", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY", "AMT_GOODS_PRICE", "NAME_TYPE_SUITE", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "REGION_POPULATION_RELATIVE", "DAYS_BIRTH", "DAYS_EMPLOYED", "DAYS_REGISTRATION", "DAYS_ID_PUBLISH", "OWN_CAR_AGE", "FLAG_MOBIL", "FLAG_EMP_PHONE", "FLAG_WORK_PHONE", "FLAG_CONT_MOBILE", "FLAG_PHONE", "FLAG_EMAIL", "OCCUPATION_TYPE", "CNT_FAM_MEMBERS", "REGION_RATING_CLIENT", "REGION_RATING_CLIENT_W_CITY", "WEEKDAY_APPR_PROCESS_START", "HOUR_APPR_PROCESS_START", "REG_REGION_NOT_LIVE_REGION", "REG_REGION_NOT_WORK_REGION", "LIVE_REGION_NOT_WORK_REGION", "REG_CITY_NOT_LIVE_CITY", "REG_CITY_NOT_WORK_CITY", "LIVE_CITY_NOT_WORK_CITY", "ORGANIZATION_TYPE", "EX